In [ ]:
import pyVoxelFEM
import MeshFEM, mesh
import numpy as np
import matplotlib.pyplot as plt
from tri_mesh_viewer import QuadHexViewer
import time
import copy
import benchmark

# Helpers
import sys
sys.path.append('./helpers')
from ipopt_helpers import initializeTensorProductSimulator, problemObjectWrapper, initializeIpoptProblem

In [ ]:
import parallelism, psutil
parallelism.set_max_num_tbb_threads(psutil.cpu_count(logical=False))
parallelism.set_hessian_assembly_num_threads(min(psutil.cpu_count(logical=False), 4))
parallelism.set_gradient_assembly_num_threads(min(psutil.cpu_count(logical=False), 8))

In [ ]:
import gzip
d1 = np.load(gzip.open('data/test_densities_3D.npy.gz', 'r'))

In [ ]:
MATERIAL_PATH = '../examples/materials/B9Creator.material'
BC_PATH = '../examples/bcs/3D/cantilever_flexion_E.bc'  # 3D cantilever configuration

orderFEM = [1, 1, 1]
domainCorners = [[0, 0, 0], [2, 1, 1]]
gridDimensions = [64, 32, 32]
E0 = 1
Emin = 1e-9
SIMPExponent = 3

# Constraints
maxVolume = 0.6
constraints = [pyVoxelFEM.TotalVolumeConstraint(maxVolume)]

# Filters: comment a line to remove the corresponding filter
filters = [
    pyVoxelFEM.SmoothingFilter(),
    pyVoxelFEM.ProjectionFilter(),
]

In [ ]:
benchmark.reset()
# Initializations
tps = initializeTensorProductSimulator(
    orderFEM, domainCorners, gridDimensions, maxVolume, E0, Emin, SIMPExponent, MATERIAL_PATH, BC_PATH
)
objective = pyVoxelFEM.MultigridComplianceObjective(tps.multigridSolver(2))
benchmark.report()

In [ ]:
top = pyVoxelFEM.TopologyOptimizationProblem(tps, objective, constraints, filters) # TO problem

In [ ]:
objective.mgSmoothingIterations = 2
objective.mgIterations = 1
objective.fullMultigrid = True
objective.cgIter = 1000

In [ ]:
import parallelism, psutil
parallelism.set_max_num_tbb_threads(psutil.cpu_count(logical=False))
parallelism.set_hessian_assembly_num_threads(min(psutil.cpu_count(logical=False), 4))
parallelism.set_gradient_assembly_num_threads(min(psutil.cpu_count(logical=False), 8))

In [ ]:
# Compute "ground truth" (E_min = 1e-9) to high accuracy
ground_truth_u = []
ground_truth_compliance = []
tps.E_min = 1e-9
objective.tol = 1e-8
for i in range(100):
    top.setVars(d1[i, :, :, :].ravel())
    ground_truth_u.append(objective.u())
    ground_truth_compliance.append(objective.compliance())

In [ ]:
E_mins = [1e-2, 1e-3, 1e-4, 1e-5, 1e-6, 1e-7, 1e-8]
def runAnalysis(cgtol):
    """
    Plot the convergence benchmark statistics for a given CG tolerance
    and return the total number of CG iterations taken across the bechmark dataset
    for each E_min setting.
    """
    objective.tol = cgtol
    
    from collections import defaultdict
    residuals = []
    u_errors = defaultdict(list)
    compliance_errors = defaultdict(list)
    residual_lens = defaultdict(list)
    objective.residual_cb = lambda it, r: residuals[-1].append(r)
    for E_min in E_mins:
        tps.E_min = E_min
        for i in range(100):
            residuals.append([])
            top.setVars(d1[i, :, :, :].ravel())
            u_errors[E_min].append(np.linalg.norm((objective.u() - ground_truth_u[i]).ravel()) / np.linalg.norm(ground_truth_u[i].ravel()))
            compliance_errors[E_min].append(abs((objective.compliance() - ground_truth_compliance[i])/ground_truth_compliance[i]))
            residual_lens[E_min].append(len(residuals[-1]))
            
    fig = plt.figure(figsize=[12,4])
    plt.subplot(1, 3, 1)
    for k in E_mins: plt.semilogy(u_errors[k][1:], label=f'E_min = {k:0.0e}')
    plt.title('Displacement error')
    plt.subplot(1, 3, 2)
    for k in E_mins: plt.semilogy(compliance_errors[k][1:], label=f'E_min = {k:0.0e}')
    plt.title('Compliance error')
    plt.subplot(1, 3, 3)
    for k in E_mins: plt.plot(residual_lens[k], label=f'E_min = {k:0.0e}')
    handles, labels = plt.gca().get_legend_handles_labels()
    plt.title('CG iterations')
    fig.legend(handles, labels)
    plt.suptitle(f'MG Solver Analysis for CG tol {cgtol}')
    plt.savefig(f'mg_analysis_tol{cgtol:0.0e}.png')
    plt.close()
    return [np.sum(residual_lens[k]) for k in E_mins]

In [ ]:
def runForZeroinit(zeroinit):
    objective.zeroInit = zeroinit
    name = "zeroInit" if zeroinit else "prevInit"
    top.setVars(d1[0, :, :, :].ravel())
    
    cg_tols = [1e-2, 1e-3, 1e-4, 1e-5, 1e-6, 1e-7, 1e-8]
    total_cg_iters = []
    for tol in cg_tols: total_cg_iters.append(runAnalysis(tol))
    plt.imsave(f'mg_analysis_summary_{name}.png', np.concatenate([plt.imread(f'mg_analysis_tol{tol}.png') for tol in cg_tols]))

    for i, tol in enumerate(cg_tols):
        plt.semilogx(E_mins, total_cg_iters[i], label=f'cg tol = {tol}')
    plt.xlabel('E_min')
    plt.ylabel('total CG iterations')
    plt.legend()
    plt.savefig(f'cg_iter_summary_{name}.png')

In [ ]:
runForZeroinit(False)

In [ ]:
runForZeroinit(True)

# Standalone benchmark runs

In [ ]:
tps.E_min = 1e-5
objective.tol = 1e-5
objective.zeroInit = False
objective.mgSmoothingIterations = 2
benchmark.reset()
for i in range(100): top.setVars(d1[i, :, :, :].ravel())
benchmark.report()